In [76]:
import pandas as pd
import numpy as np
import netCDF4 as nc

In [77]:
# 处理 CSV 文件的函数
def process_csv(file_path):
    # 读取文件并剔除异常数据
    df = pd.read_csv(file_path,encoding='gb2312')
    df = df[['天','时','分', '秒','经度','纬度','空气温度','大气压强','空气湿度','绝对风速','绝对风向','10分钟平均绝对风速','10分钟平均绝对风向','CTD深度','CTD温度','CTD盐度']]
    # 提取文件名的时间
    name = file_path.split('\\')[-1] 
    extract_time = name.split('_')[-1][:12] # '202405091735'
    hour = int(df['时'].iloc[0]); minute = int(df['分'].iloc[0]); second = int(df['秒'].iloc[0])
    hour = '{:02d}'.format(hour)
    minute = '{:02d}'.format(minute)
    second = '{:02d}'.format(second)
    time = extract_time[:8] + hour + minute + second
    df['时间'] = time
    index_temp = (df['CTD温度'] < 40.0) &(df['CTD温度'] > -2.5)
    df = df[index_temp]
    index_salt = (df['CTD盐度'] < 41.0) &(df['CTD盐度'] > 0)
    df = df[index_salt]
    index_pre = df['CTD深度'] > 0
    df = df[index_pre]
    return df

In [78]:
file_path = r'C:\Users\lenovo\Desktop\WPV\WPV_WP_001_202405091735.csv'
df = process_csv(file_path)


In [79]:
df

,天,时,分,秒,经度,纬度,空气温度,大气压强,空气湿度,绝对风速,绝对风向,10分钟平均绝对风速,10分钟平均绝对风向,CTD深度,CTD温度,CTD盐度,时间
0,1,9,20,11,117.067902,39.098915,14,1,0,0.3,62.5,56,23.98,100,25,35,20240509092011


In [80]:
df['时间'].iloc[0]

'20240509092011'

In [81]:
str(df['时间'].iloc[0])

'20240509092011'

In [82]:
def nc_generate(df,new_path,file_path,lon2 = 105.21,lat2 = 15.11,ReleaseTime = '2022-12-11 14:03:07 UTC',ReleaseLongitude = '111.8444°E',ReleaseLatitude = '17.4169°N',Comment = 'quality control data',information ='') :
    # WPV_Type:'HZZ' lon2:105.21 lat2:15.11 deploy_time:202303270530(整数就可以，字符串也可以)
    name = file_path.split('\\')[-1] # 'WPV_HYY_003_202303270530.csv'
    WPV_Type = name.split('_')[1]
    deviceID = name.split('_')[2]
    
    time = df['时间']
    lon = round(df['经度'].iloc[-1],4)
    lat = round(df['纬度'].iloc[-1],4) 
    ssat = round(df['空气温度'],2)
    ssap = round(df['大气压强'],4)
    rh = round(df['空气湿度'],4)
    true_wind_speed = round(df['绝对风速'],2)
    true_wind_dir = round(df['绝对风向'],2)
    true_wind_speed_average = round(df['10分钟平均绝对风速'],2)
    true_wind_dir_average = round(df['10分钟平均绝对风向'],2)
    seawater_pressure = round(df['CTD深度'],3)
    seawater_temperature = round(df['CTD温度'],4)
    seawater_salinity = round(df['CTD盐度'],4)
    import seawater
    seawater_density = seawater.dens(seawater_salinity, seawater_temperature, seawater_pressure)
    seawater_sound_velocity = seawater.svel(seawater_salinity, seawater_temperature, seawater_pressure)
    seawater_density = np.round(seawater_density,4); seawater_sound_velocity = np.round(seawater_sound_velocity,4); 

    num_row = df.shape[0] # 获取数据行数，为 node_dim准备
    # 创建nc文件
    format = 'NETCDF3_CLASSIC'
    last_time = str(time[0])[:12]
    filename = f'WPV_{WPV_Type}_{deviceID}_{lon2}E_{lat2}N_{last_time}_Q.nc'
    file  = nc.Dataset(new_path+'/'+f'{filename}', 'w',format= format)

    # 创建维度
    node_dim = file.createDimension('num_node',num_row)
    lon_dim = file.createDimension('num_longitude',1)
    lat_dim = file.createDimension('num_latitude',1)
    time_dim = file.createDimension('num_time',num_row)

    # 创建变量
    time_var = file.createVariable('time','f8',('num_time',))
    lon_var = file.createVariable('longitude','f4',('num_longitude',))
    lat_var = file.createVariable('latitude','f4',('num_latitude',))
        # 'Latitude': 这是新创建的变量的名称
        # 'f4': 这指定了变量的数据类型。'f8'代表64位浮点数（双精度）
        # ('Latitude',):指定新变量所依赖的维度，('Latitude',)创建元组，而 ('Latitude')为一个字符串
    ssat_var = file.createVariable('ssat','f4',('num_node',))
    ssap_var = file.createVariable('ssap','f4',('num_node',))
    rh_var = file.createVariable('rh','f4',('num_node',))
    true_wind_speed_var = file.createVariable('true_wind_speed','f4',('num_node',))
    true_wind_dir_var = file.createVariable('true_wind_dir','f4',('num_node',))
    true_wind_speed_average_var = file.createVariable('true_wind_speed_average','f4',('num_node',))
    true_wind_dir_average_var = file.createVariable('true_wind_dir_average','f4',('num_node',))
    seawater_pressure_var = file.createVariable('seawater_pressure','f4',('num_node',))
    seawater_temperature_var = file.createVariable('seawater_temperature','f4',('num_node',))
    seawater_salinity_var = file.createVariable('seawater_salinity','f4',('num_node',))
    seawater_density_var = file.createVariable('seawater_density','f4',('num_node',))
    seawater_sound_velocity_var = file.createVariable('seawater_sound_velocity','f4',('num_node',))
    # 添加属性
    time_var.long_name="the time observed by the WPV"
    lon_var.long_name = 'the longitude located by the WPV' 
    lon_var.units = "degrees_east" 
    lat_var.long_name = 'the latitude located by the WPV'
    lat_var.units = "degrees_north"
    ssat_var.long_name = "the sea surface air temperature observed by the WPV"
    ssat_var.units = "Celsius"
    ssap_var.long_name = "the sea surface air pressure observed by the WPV"
    ssap_var.units = "kPa" 
    rh_var.long_name = "the sea surface humidity observed by the WPV"
    rh_var.units = "%" 
    true_wind_speed_var.long_name =  "the sea surface true wind speed observed by the WPV"
    true_wind_speed_var.units = "m/s" 
    true_wind_dir_var.long_name = "the sea surface true wind direction observed by the WPV"
    true_wind_dir_var.units = "degree" 
    true_wind_speed_average_var.long_name = 'the sea surface average true wind speed in ten minutes observed by the WPV'
    true_wind_speed_average_var.units = "m/s" 
    true_wind_dir_average_var.long_name = 'the sea surface average true wind direction in ten minutes observed by the WPV'
    true_wind_dir_average_var.units = "degree"
    seawater_pressure_var.long_name = 'the seawater pressure observed by the WPV'
    seawater_pressure_var.units = "dbar"
    seawater_temperature_var.long_name = 'the seawater temperature observed by the WPV'
    seawater_temperature_var.units = "Celsius"
    seawater_salinity_var.long_name = 'the seawater salinity observed by the WPV'
    seawater_salinity_var.units = "PSU"
    seawater_density_var.long_name = 'the seawater density observed by the WPV'
    seawater_density_var.units = "kg/m3"
    seawater_sound_velocity_var.long_name = 'the seawater sound velocity observed by the WPV'
    seawater_sound_velocity_var.units = "m/s"
    
    # 变量中写入数据
        # 必须time_var[:] = time_data，不可以time_var= time_data
        # 使用createVariable方法创建一个变量后，这个变量实际上是一个特殊的对象，它代表netCDF文件中的一个数据区域。
        # time_var = time_data会改变time_var的引用，而不是修改它所代表的数据区域的内容
        # 相反，time_var[:]是用来引用该变量整个数据区域的一个“切片”
        # time_var[:] = time_data 实际上是将time_data数组中的数据复制到time_var所代表的数据区域中
    time_var[:] = time
    lon_var[:] = lon
    lat_var[:] = lat
    ssat_var[:] = ssat
    ssap_var[:] = ssap
    rh_var[:] = rh
    true_wind_speed_var[:] = true_wind_speed
    true_wind_dir_var[:] = true_wind_dir
    true_wind_speed_average_var[:] = true_wind_speed_average
    true_wind_dir_average_var[:] = true_wind_dir_average
    seawater_pressure_var[:] = seawater_pressure
    seawater_temperature_var[:] = seawater_temperature
    seawater_salinity_var[:] = seawater_salinity
    seawater_density_var[:] = seawater_density
    seawater_sound_velocity_var[:] = seawater_sound_velocity

    # 全局属性(需要放到最后来，不然添加不上全局属性)
    file.WPV_SN = WPV_Type + '_' + deviceID
    file.WPV_ReleaseTime = ReleaseTime
    file.WPV_ReleaseLongitude = ReleaseLongitude
    file.WPV_ReleaseLatitude = ReleaseLatitude
    file.Sensor_accuracy = ''
    file.Comment = Comment
    file.Information = information
    file.Manufacturer = '天津大学'
    file.Source = 'Wei Ma, TianJin University, wei.ma@tju.edu.cn'
    import datetime
    import pytz
    current_time = datetime.datetime.now() # 获取当前时间
    utc_time = current_time.astimezone(pytz.utc) # 将当前时间转换为 UTC 时间
    formatted_utc_time = utc_time.strftime("%Y-%m-%d %H:%M:%S %Z") # 格式化并显示 UTC 时间
    time_now = str(formatted_utc_time)[:19]
    file.History = f'Created by Python at {time_now} UTC'

    file.close() # 关闭文件
    nc_path = new_path+'/'+f'{filename}'
    return nc_path.replace('\\', '/')

In [83]:
file_path = r'C:\Users\lenovo\Desktop\WPV\WPV_WP_001_202405091735.csv'
df = process_csv(file_path=file_path)
# new_path = r'C:\Users\lenovo\Desktop\java\Glider05'
new_path = r'C:\Users\lenovo\Desktop\WPV' #'C:/Users/lenovo/Desktop/java/Glider05'  'C:/Users/lenovo/Desktop/任务帧nc数据'
Glider_Type = 'HY-BX' ;lon2 = 105.21 ;lat2 = 15.11 
nc_generate(df,new_path,file_path,lon2,lat2)

'C:/Users/lenovo/Desktop/WPV/WPV_WP_001_105.21E_15.11N_202405090920_Q.nc'